In [1]:
!pip install datasets gdown
!pip install torch einops
!pip install 'numpy<2'
!pip install -U sentence-transformers
!pip install 'accelerate>=0.26.0'
!pip install gdown
!pip install -q pyvi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 81.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 89.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.0 MB/s eta 0:00:0000:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 93.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!gdown --id 1GuWOFMPim-KM0KS_CHSfrb-tkUYWo-io --output train.zip
#https://drive.google.com/file/d/1GuWOFMPim-KM0KS_CHSfrb-tkUYWo-io/view?usp=drive_link
#!unzip train.zip 

/opt/conda/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1GuWOFMPim-KM0KS_CHSfrb-tkUYWo-io
From (redirected): https://drive.google.com/uc?id=1GuWOFMPim-KM0KS_CHSfrb-tkUYWo-io&confirm=t&uuid=9727b3b2-64f3-448a-af15-3ef973e064ea
To: /train.zip
100%|██████████████████████████████████████| 45.5M/45.5M [00:01<00:00, 24.7MB/s]


In [4]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments, datasets
from torch.utils.data import DataLoader
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.datasets import NoDuplicatesDataLoader
from sklearn.model_selection import train_test_split
from pyvi.ViTokenizer import tokenize


from datasets import Dataset
import pandas as pd
import ast

In [5]:
final = pd.read_csv("train_bi_2.csv")
final.head()

,query,answer,cid,qid
0,Người học ngành quản_lý khai_thác công_trình t...,"Khả_năng học_tập , nâng cao_trình_độ - Khối_lư...",62492,161615
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,151154,80037
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"Điều 20 . Ưu_đãi về thời_gian miễn thuế , gi...",75071,124074
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"Điều_kiện được hưởngCán bộ quản_lý , giáo_viên...",225897,146841
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,"Nguyên_tắc áp dụng 1 . Trường_hợp công_chức ,...",68365,6176


In [6]:
train_data, eval_data = train_test_split(final, test_size=0.1, random_state=42)
print(f"Train size: {len(train_data)}, Eval size: {len(eval_data)}")
final.head()

Train size: 110688, Eval size: 12299


,query,answer,cid,qid
0,Người học ngành quản_lý khai_thác công_trình t...,"Khả_năng học_tập , nâng cao_trình_độ - Khối_lư...",62492,161615
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,151154,80037
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"Điều 20 . Ưu_đãi về thời_gian miễn thuế , gi...",75071,124074
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"Điều_kiện được hưởngCán bộ quản_lý , giáo_viên...",225897,146841
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,"Nguyên_tắc áp dụng 1 . Trường_hợp công_chức ,...",68365,6176


In [7]:
def create_contrastive_dataset(filtered_data):

    dataset_dict = {
        "anchor": [],          # câu query
        "positive": [],       # positive document
    }
    c = 0
    for index, row in filtered_data.iterrows():
        query = row['query']
        item_pos = row['answer']
        dataset_dict["anchor"].append(query)
        dataset_dict["positive"].append(item_pos)

    dataset = Dataset.from_dict(dataset_dict)
    return dataset

train_dataset = create_contrastive_dataset(train_data)
print(len(train_dataset))
validation_dataset = create_contrastive_dataset(eval_data)
print(len(validation_dataset))


110688
12299


In [8]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from transformers import AdamW


model = SentenceTransformer(
    "bkai-foundation-models/vietnamese-bi-encoder",
    trust_remote_code=True,
)
model = model.to('cuda')
loss = losses.CachedMultipleNegativesRankingLoss(model, mini_batch_size = 1024)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [ ]:
args = SentenceTransformerTrainingArguments(
    # Required parameter
    output_dir="models/BKAI",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_ratio=0.1,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # Ensures no duplicates in the batch
    learning_rate=2e-05,  # Correct parameter for learning rate
    save_strategy="steps",
    save_steps=300,
    save_total_limit=2,
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=300,
    metric_for_best_model="eval_loss",
    warmup_steps=50,
    fp16=True,
    weight_decay=0.01,  # Correctly added weight decay
    load_best_model_at_end=True
)

# Use AdamW optimizer
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
from transformers import EarlyStoppingCallback
from transformers import get_scheduler
num_samples = len(train_dataset)
batch_size = args.per_device_train_batch_size
num_epochs = args.num_train_epochs
num_training_steps = num_samples // batch_size * num_epochs



early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,  # Dừng sau 3 lần validation loss không cải thiện
    early_stopping_threshold=0.005  # Ngưỡng cải thiện nhỏ nhất
)

scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(len(train_dataset) * 0.1),
    num_training_steps=num_training_steps
)


trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    loss=loss,
    optimizers=(optimizer, scheduler)
)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [11]:
#8
#4715ae4bb71f935c17bcc9e2278c5ac7a8c116c4
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.00 GiB. GPU 0 has a total capacity of 31.73 GiB of which 2.28 GiB is free. Process 1201342 has 29.45 GiB memory in use. Of the allocated memory 23.07 GiB is allocated by PyTorch, and 6.00 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [20]:
model_save_path = "bi_encoder_model"
model.save(model_save_path)

In [19]:
print("a")

a


In [ ]:
import pandas as pd
pd.DataFrame(trainer.state.log_history)

In [ ]:
df = pd.DataFrame(trainer.state.log_history)

# Save the DataFrame to a CSV file named 'training_log.csv'
df.to_csv('training_log.csv', index=False)

In [ ]:
print("a")